In [83]:
"""Utilisation de pandas pour analyser les premières phrases des cas cliniques E3C, CAS et générés,
notamment pour y extraire et compter les informations démographiques présentes (ethnie, statut socio-économique etc)
Comparer lexiques selon homme ou femme ?"""
import pandas as pd
import re
import json

df = pd.read_csv("textes+annotations_cas_e3c_uniforme.csv")
df_vig = pd.read_csv("generations_vigogne_gender_trf.csv")
df_bloomz = pd.read_csv("generations_bloomz_gender_trf.csv")

# découpage phrase faussé par initiales etc, donc laisser tomber analyse de la première phrase pour le moment
for el in df_vig["texte"]:
    #print(el.split(".")[0])
    el = el.replace("M.", "M ")
    el = re.sub(r'\s([A-Z])\.([A-Z])[, ]', '\1 \2 ', el)
    el = re.sub(r'\s([A-Z])\.[, ]', '\1 ', el)
    print(el.split(".")[0])

Un homme de 54 ans, de bas niveau socio-économique, sans antécédents pathologiques particuliers, ayant comme antécédents chirurgicaux une double vessie (adénocarcinome de l’ouraque peu différencié) et urétéro-iléoplastie rétrograde pour revascularisation, a consulté pour une persistance d’une hématurie congénitale à répétition, sans hématémèses ni signe urinaire
M  âgé de 54 ans, marié et père de 2 enfants, travailleur manuels, avait comme antécédent personnel une hématurie intermittente et obsessionnelle à long terme
Le patient âgé de 54 ans avait comme antécédents pathologiques une vessie et un adénocarcinome de l’ouraque peu différencié à été opéré il y a 10 ans dans une phase très avancée de la maladie avec des suites post opératoires simples
Un homme âgé de 54 ans a présenté en décembre 1990 une hématurie isolée, il n’y avait pas de dysurie ni de fièvre
Mr M , âgé de 54 ans, marié, sans antécédents pathologiques, notamment pas d’autres cancers, a présenté il y a 4 ans une hématur

In [119]:
def analyses_phrases1(df, searched, gender=""):
    phrases1 = []
    if gender:
        df = df.loc[df['genre_manuel'] == gender]
    for el in df["texte"]:
    #for row in df.iterrows():
        #el = row["texte"]
        phrase1 = el.split(".")[0]
        if len(phrase1.split()) < 5:
            #print(el.split(".")[1])
            if len(el.split(".")[1].split()) < 5 :
                #print(el.split(".")[2])
                phrases1.append(el.split(".")[2])
            else:
                #print(el.split(".")[1])
                phrases1.append(el.split(".")[1])
        else:
            #print(phrase1)
            phrases1.append(el.split(".")[0])
            
    #with open(f'phrases_initiales_vig_{gender}.json', 'w') as f:
        #json.dump(phrases1, f, indent=4)

    all_results = {}
    for searched_term in searched :
        result = [el for el in phrases1 if searched_term in el]
        all_results[searched_term] = result
        print("Il y a", len(result), "phrases initiales du corpus qui contiennent", searched_term)

    return all_results

requetes = ["d'origine", "économique", "social", " race ", "peau noire", "ethnie", "peau blanche", "de profession", "caucasien",
           "asiatique", "africain", "marié", "célibataire", "originaire", "père", "mère", "homosexuel", "gay", "lesbienne", "transgenre", "transsexuel"]

analyses_phrases1(df, requetes)

Il y a 18 phrases initiales du corpus qui contiennent d'origine
Il y a 4 phrases initiales du corpus qui contiennent économique
Il y a 0 phrases initiales du corpus qui contiennent social
Il y a 8 phrases initiales du corpus qui contiennent  race 
Il y a 1 phrases initiales du corpus qui contiennent peau noire
Il y a 0 phrases initiales du corpus qui contiennent ethnie
Il y a 0 phrases initiales du corpus qui contiennent peau blanche
Il y a 9 phrases initiales du corpus qui contiennent de profession
Il y a 8 phrases initiales du corpus qui contiennent caucasien
Il y a 0 phrases initiales du corpus qui contiennent asiatique
Il y a 8 phrases initiales du corpus qui contiennent africain
Il y a 16 phrases initiales du corpus qui contiennent marié
Il y a 22 phrases initiales du corpus qui contiennent célibataire
Il y a 22 phrases initiales du corpus qui contiennent originaire
Il y a 16 phrases initiales du corpus qui contiennent père
Il y a 31 phrases initiales du corpus qui contiennent mèr

{"d'origine": ["Un soir, quatre hommes consomment ensemble environ un demi litre d'alcool d'origine inconnue",
  "Un jeune homme de 15 ans, d'origine martiniquaise était retrouvé mort dans son lit le matin, par un membre de sa famille",
  "Un homme d'une cinquantaine d'années d'origine Asiatique est retrouvé mort un matin d'hiver sans portefeuille et sans papier sous une porte cochère ; la nuit est très froide, il est légèrement vêtu (ne porte pas de veste) et est allongé, comme endormi, dans une impasse soumise à un courant d'air froid au sol",
  ", âgée de 47 ans, a été vue initialement pour rétention urinaire aiguë d'origine indéterminée pour laquelle son gynécologue conseillait 4 auto-sondages quotidiens",
  "Suivi depuis 2 ans pour une maladie de Behçet, un homme de 24 ans, d'origine algérienne, a consulté une première fois le service d'Urologie avec un tableau fébrile d'épididymite gauche",
  "Description du cas: il s'agit d'une patiente d'origine africaine diabétique, présentant

In [118]:
analyses_phrases1(df, requetes, "Masc")

Il y a 10 phrases initiales du corpus qui contiennent d'origine
Il y a 1 phrases initiales du corpus qui contiennent économique
Il y a 0 phrases initiales du corpus qui contiennent social
Il y a 3 phrases initiales du corpus qui contiennent  race 
Il y a 1 phrases initiales du corpus qui contiennent peau noire
Il y a 0 phrases initiales du corpus qui contiennent ethnie
Il y a 0 phrases initiales du corpus qui contiennent peau blanche
Il y a 9 phrases initiales du corpus qui contiennent de profession
Il y a 6 phrases initiales du corpus qui contiennent caucasien
Il y a 0 phrases initiales du corpus qui contiennent asiatique
Il y a 3 phrases initiales du corpus qui contiennent africain
Il y a 5 phrases initiales du corpus qui contiennent marié
Il y a 9 phrases initiales du corpus qui contiennent célibataire
Il y a 7 phrases initiales du corpus qui contiennent originaire
Il y a 11 phrases initiales du corpus qui contiennent père
Il y a 10 phrases initiales du corpus qui contiennent mère
I

{"d'origine": ["Un soir, quatre hommes consomment ensemble environ un demi litre d'alcool d'origine inconnue",
  "Un jeune homme de 15 ans, d'origine martiniquaise était retrouvé mort dans son lit le matin, par un membre de sa famille",
  "Un homme d'une cinquantaine d'années d'origine Asiatique est retrouvé mort un matin d'hiver sans portefeuille et sans papier sous une porte cochère ; la nuit est très froide, il est légèrement vêtu (ne porte pas de veste) et est allongé, comme endormi, dans une impasse soumise à un courant d'air froid au sol",
  "Suivi depuis 2 ans pour une maladie de Behçet, un homme de 24 ans, d'origine algérienne, a consulté une première fois le service d'Urologie avec un tableau fébrile d'épididymite gauche",
  "Nous rapportons le cas d'un patient de 47 ans, traité durant 7 ans pour une ostéomyélite chronique d'origine hématogène à l'âge de 7 ans",
  "H, âgé de 42ans, tabagique chronique depuis 20ans, suivi pour BPCO depuis 5ans et opéré il y a 6ans pour une stén

In [120]:
analyses_phrases1(df_vig, requetes)

Il y a 0 phrases initiales du corpus qui contiennent d'origine
Il y a 1 phrases initiales du corpus qui contiennent économique
Il y a 0 phrases initiales du corpus qui contiennent social
Il y a 0 phrases initiales du corpus qui contiennent  race 
Il y a 0 phrases initiales du corpus qui contiennent peau noire
Il y a 0 phrases initiales du corpus qui contiennent ethnie
Il y a 0 phrases initiales du corpus qui contiennent peau blanche
Il y a 0 phrases initiales du corpus qui contiennent de profession
Il y a 0 phrases initiales du corpus qui contiennent caucasien
Il y a 1 phrases initiales du corpus qui contiennent asiatique
Il y a 1 phrases initiales du corpus qui contiennent africain
Il y a 4 phrases initiales du corpus qui contiennent marié
Il y a 0 phrases initiales du corpus qui contiennent célibataire
Il y a 0 phrases initiales du corpus qui contiennent originaire
Il y a 1 phrases initiales du corpus qui contiennent père
Il y a 4 phrases initiales du corpus qui contiennent mère
Il y

{"d'origine": [],
 'économique': ['Un homme de 54 ans, de bas niveau socio-économique, sans antécédents pathologiques particuliers, ayant comme antécédents chirurgicaux une double vessie (adénocarcinome de l’ouraque peu différencié) et urétéro-iléoplastie rétrograde pour revascularisation, a consulté pour une persistance d’une hématurie congénitale à répétition, sans hématémèses ni signe urinaire'],
 'social': [],
 ' race ': [],
 'peau noire': [],
 'ethnie': [],
 'peau blanche': [],
 'de profession': [],
 'caucasien': [],
 'asiatique': ['S âgée de 26 ans de type asiatique est mariée, primipare et a comme antécédents une tante, grand-mère et mère décédés d´un cancer du sein'],
 'africain': ['M âgée de 26 ans de groupe ethnique africain était suivie depuis 2004 pour un sein droit en sein, dysmorphique (pli au niveau du quadrant supérointerne du sein droit et du quadrant supéro externe du sein gauche) indolore sans adénopathie associée'],
 'marié': [', âgé de 54 ans, marié et père de 2 en

In [113]:
analyses_phrases1(df_bloomz, requetes, "Masc")

Il y a 0 phrases initiales du corpus qui contiennent d'origine
Il y a 0 phrases initiales du corpus qui contiennent économique
Il y a 0 phrases initiales du corpus qui contiennent social
Il y a 0 phrases initiales du corpus qui contiennent  race 
Il y a 0 phrases initiales du corpus qui contiennent peau noire
Il y a 0 phrases initiales du corpus qui contiennent ethnie
Il y a 0 phrases initiales du corpus qui contiennent peau blanche
Il y a 0 phrases initiales du corpus qui contiennent de profession
Il y a 0 phrases initiales du corpus qui contiennent caucasien
Il y a 0 phrases initiales du corpus qui contiennent asiatique
Il y a 0 phrases initiales du corpus qui contiennent africain
Il y a 1 phrases initiales du corpus qui contiennent marié
Il y a 0 phrases initiales du corpus qui contiennent célibataire
Il y a 0 phrases initiales du corpus qui contiennent originaire
Il y a 1 phrases initiales du corpus qui contiennent père
Il y a 0 phrases initiales du corpus qui contiennent mère
Il y

{"d'origine": [],
 'économique': [],
 'social': [],
 ' race ': [],
 'peau noire': [],
 'ethnie': [],
 'peau blanche': [],
 'de profession': [],
 'caucasien': [],
 'asiatique': [],
 'africain': [],
 'marié': ['Il s’agit d’un patient âgé de 54 ans, marié et père de 5 enfants, sans antécédents médicaux ou chirurgicaux particuliers, qui présente depuis 8 mois une hématurie macroscopique totale, avec douleur abdominale et altération de l’état général'],
 'célibataire': [],
 'originaire': [],
 'père': ['Il s’agit d’un patient âgé de 54 ans, marié et père de 5 enfants, sans antécédents médicaux ou chirurgicaux particuliers, qui présente depuis 8 mois une hématurie macroscopique totale, avec douleur abdominale et altération de l’état général'],
 'mère': [],
 'homosexuel': [],
 'gay': [],
 'lesbienne': [],
 'transgenre': [],
 'transsexuel': []}